In [3]:
ex = "RRID:SCR_010479"
id = ex.find(":")
print(ex[0:id])
print(ex[id+1:])

RRID
SCR_010479


In [ ]:
import glob
import xml.etree.ElementTree as ET

repo_list = glob.glob('../data/re3dataRecords/*.xml')

entries = ET.Element("registryEntries")


for repo in repo_list:
    root = ET.parse(repo).getroot()
    id = root.find('.//{http://www.re3data.org/schema/2-2}re3data.orgIdentifier')
    if id is None or id.text == '':
        continue
    entry = ET.Element("registryEntry")
    ET.SubElement(entry, 'internalIdentifier').text = id
    name =  root.find('.//{http://www.re3data.org/schema/2-2}repositoryName')
    if not name is None :
        if name.text != '':
            if 'language' in name.attrib and name.attrib['language']!= '':
                ET.SubElement(entry, "name", nameLanguage = name.attrib['language'], languageVocabulary = "ISO 639-2").text = name.text
            else:
                ET.SubElement(entry, "name").text = name.text
    additionalname = root.find('.//{http://www.re3data.org/schema/2-2}additionalName')
    if not additionalname is None:
        if additionalname.text != '':
            if 'language' in additionalname.attrib and additionalname.attrib['language'] != '':
                ET.SubElement(entry, "name", nameLanguage = additionalname.attrib['language'], languageVocabulary = "ISO 639-2").text = additionalname.text
            else:
                ET.SubElement(entry,'additionalName').text = additionalname.text
    repoUrl = root.find('.//{http://www.re3data.org/schema/2-2}repositoryURL')
    if not repoUrl is None and repoUrl.text != '':
        ET.SubElement(entry,'url').text = repoUrl.text
    type = root.find('.//{http://www.re3data.org/schema/2-2}type')
    if not type is None and type.text != '':
        ET.SubElement(entry,'type').text = type.text
    dex =  root.find('.//{http://www.re3data.org/schema/2-2}description')
    if not dex is None and dex.text != '':
        ET.SubElement(entry,'description').text = dex.text
    ctype = root.findall('.//{http://www.re3data.org/schema/2-2}contentType')
    if len( ctype ) > 0:
        for ct in ctype:
            ET.SubElement(entry,'content').text = ct.text
    repoId =  root.findall('.//{http://www.re3data.org/schema/2-2}repositoryIdentifier')
    if len( repoId ) > 0 :
        for repo in repoId:
            splitIndex  = repo.text.find(':')
            pid = repo.text[0:splitIndex]
            value = repo.text[splitIndex+1:]
            ET.SubElement(entry, "identifier", type = pid).text = value
    subjs = root.findall('.//{http://www.re3data.org/schema/2-2}subject')
    if len(subjs) > 0:
        for s in subjs:
            if s.text != '':
                ET.SubElement(entry,'subject').text = s.text
    kws = root.findall('.//{http://www.re3data.org/schema/2-2}keyword')
    if len(kws) > 0:
        for k in kws:
            if k.text != '':
                ET.SubElement(entry,'keyword').text = k.text

    if 'acronym' in name:
  
        if 'url' in org and org['url'] != '':
            
        if 'identifiers' in org and org['identifiers'] != []:
            for ide in org['identifiers']:
                if 'type' in ide:
                    ET.SubElement(organization,"id", type = ide['type']).text = ide['identifier'] 
                else: 
                    ET.SubElement(organization,"id").text = ide['identifier']            
    ists = root.findall('.//{http://www.re3data.org/schema/2-2}institution')
    
    if len(ists) > 0:

        for i in ists:
            iname = i.find('.//{http://www.re3data.org/schema/2-2}institutionName') 
            if iname is not None:
                if iname.text != '':
                    organization = ET.Element("organization")
                    if 'language' in iname.attrib and iname.attrib['language'] != '':
                        ET.SubElement(organization,"name", nameLanguage = iname.attrib['language'], languageVocabulary = "ISO 639-2").text = iname.text
                    else:
                        ET.SubElement(organization,"name").text = iname.text
                    country = i.find('.//{http://www.re3data.org/schema/2-2}institutionCountry')
                    if  country is not None and  country.text != '':
                        ET.SubElement(organization,"country").text = country.text
                    homepage = i.find('.//{http://www.re3data.org/schema/2-2}institutionURL')
                    if homepage is not None and homepage.text != '':
                        ET.SubElement(organization,"organizationUrl").text = homepage.text
                    identifiers = i.findall('.//{http://www.re3data.org/schema/2-2}institutionIdentifier')
                    if len(identifiers) > 0:
                        for identifier in identifiers:
                            if identifier is not None and identifier.text != '':
                                ET.SubElement(organization,"id").text = ide['identifier']   
            entry.append(organization)
        
    software = root.findall('.//{http://www.re3data.org/schema/2-2}software')
    if len(software) > 0:
        for s in software:
            if s.find('.//{http://www.re3data.org/schema/2-2}softwareName') is not None and s.find('.//{http://www.re3data.org/schema/2-2}softwareName').text != '':
                ET.SubElement(entry,'softwareName').text = s.find('.//{http://www.re3data.org/schema/2-2}softwareName').text != ''
        
    versioning = root.find('.//{http://www.re3data.org/schema/2-2}versioning')
    if versioning is not None and versioning.text != '':
        ET.SubElement(entry, "versioning").text = "True" 
    api = root.findall('.//{http://www.re3data.org/schema/2-2}api')
    if len(api) > 0:
        for a in api:
            if a.text != '':
                if 'apiType' in a.attrib and a.attrib['apiType'] != '':
                    ET.SubElement(entry,'apiUrl', type=a.attrib['apiType']).text = a.text
                    break

    
    #TO CONTINUE FROM HERE
    startDate = root.find('.//{http://www.re3data.org/schema/2-2}startDate')
    if not startDate is None and startDate.text != '':
        r3_coverage['startDate'] += 1
    lastUpdate = root.find('.//{http://www.re3data.org/schema/2-2}lastUpdate')
    if not lastUpdate is None and lastUpdate.text != '':
        r3_coverage['lastUpdate'] += 1
    #questo vale perche' se esiste il campo policy allora sono mandatory sia policyName che policyUrl
    policy = root.findall('.//{http://www.re3data.org/schema/2-2}policy')
    # r3_coverage['policyName'] += len(policy)
    # r3_coverage['policyUrl'] += len(policy)
    r3_coverage['policyName'] += 1#len(policy)
    r3_coverage['policyUrl'] += 1#len(policy)
    dbAccess = root.find('.//{http://www.re3data.org/schema/2-2}databaseAccess')
    if dbAccess is not None :
        r3_coverage['databaseAccessType'] += 1
        if dbAccess.find('.//{http://www.re3data.org/schema/2-2}databaseAccessType').text =='restricted':
            dbRestricted += 1
        if dbAccess.find('.//{http://www.re3data.org/schema/2-2}databaseAccessRestriction') is not None :
            r3_coverage['databaseAccessRestriction'] += 1
    dataUpload = root.find('.//{http://www.re3data.org/schema/2-2}dataUpload')
    if dataUpload is not None:
        r3_coverage['dataUploadType'] += 1
        if dataUpload.find('.//{http://www.re3data.org/schema/2-2}dataUploadType').text == 'restricted':
            dataRestrictied += 1
        if dataUpload.find('.//{http://www.re3data.org/schema/2-2}dataUploadRestriction') is not None :
            r3_coverage['dataUploadRestriction'] += 1
    dbLicense = root.findall('.//{http://www.re3data.org/schema/2-2}databaseLicense')
    if dbLicense is not None:
        r3_coverage['databaseLicenseName'] += 1
        r3_coverage['databaseLicenceUrl'] += 1
    dataLicence = root.findall('.//{http://www.re3data.org/schema/2-2}dataUploadLicense')
    if dataLicence is not None:
        r3_coverage['dataUploadLicenceName'] += 1
        r3_coverage['dataUploadLicenceUrl'] += 1

